In [18]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./myapi.db"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

naming_convention = {
    "ix": 'ix_%(column_0_label)s',
    "uq": "uq_%(table_name)s_%(column_0_name)s",
    "ck": "ck_%(table_name)s_%(column_0_name)s",
    "fk": "fk_%(table_name)s_%(column_0_name)s_%(referred_table_name)s",
    "pk": "pk_%(table_name)s"
}
Base.metadata = MetaData(naming_convention=naming_convention)

In [16]:
from sqlalchemy import Column, Integer, String, Text, DateTime, ForeignKey, Table
from sqlalchemy.orm import relationship

question_voter = Table(
    'question_voter',
    Base.metadata,
    Column('user_id', Integer, ForeignKey('user.id'), primary_key=True),
    Column('question_id', Integer, ForeignKey('question.id'), primary_key=True)
)

answer_voter = Table(
    'answer_voter',
    Base.metadata,
    Column('user_id', Integer, ForeignKey('user.id'), primary_key=True),
    Column('answer_id', Integer, ForeignKey('answer.id'), primary_key=True)
    
)

class Question(Base):
    __tablename__ = "question"
    
    id = Column(Integer, primary_key=True)
    subject = Column(String, nullable=False)
    content = Column(Text, nullable=False)
    create_date = Column(DateTime, nullable=False)
    user_id = Column(Integer, ForeignKey('user.id'), nullable=True)
    user = relationship("User", backref="question_users")
    modify_date = Column(DateTime, nullable=True)
    voter = relationship('User', secondary=question_voter, backref='question_voters')

class Answer(Base):
    __tablename__ = "answer"

    id = Column(Integer, primary_key=True)
    content = Column(Text, nullable=False)
    create_date = Column(DateTime, nullable=False)
    question_id = Column(Integer, ForeignKey("question.id"))
    question = relationship("Question", backref="answers")
    user_id = Column(Integer, ForeignKey('user.id'), nullable=True)
    user = relationship("User", backref="answer_users")
    modify_date = Column(DateTime, nullable=True)
    voter = relationship('User', secondary=answer_voter, backref='answer_voters')

class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    username = Column(String, unique=True, nullable=False)
    password = Column(String, nullable=False)
    email = Column(String, unique=True, nullable=False)

In [3]:
db = SessionLocal()

In [4]:
from datetime import datetime

In [16]:
q = Question(subject='pybo가 무엇인가요?', content='pybo에 대해서 알고 싶습니다.', create_date=datetime.now())

db.add(q)
db.commit()

In [ ]:
q.id

In [17]:

q = Question(subject='FastAPI 모델 질문입니다.', content='id는 자동으로 생성되나요?', create_date=datetime.now())
db.add(q)
db.commit()


In [11]:
for q in db.query(Answer).all():
    db.delete(q)
    db.commit()

In [12]:
db.query(Answer).all()

[]

In [13]:
db.query(Question).all()

[]

In [ ]:
db.query(Question).filter(Question.id==1).all()

In [ ]:
db.query(Question).get(1)

In [ ]:
db.query(Question).filter(Question.subject.like('%FastAPI%')).all()

In [23]:
q = db.query(Question).get(2)
q.content

'id는 자동으로 생성되나요?'

In [4]:
q.subject = "FastAPI Model Question"
db.commit()

In [18]:
q = db.query(Question).get(2)
q.subject

'FastAPI 모델 질문입니다.'

In [19]:
q = db.query(Question).get(1)
db.delete(q)
db.commit()

In [20]:
db.query(Question).all()

In [10]:
q = db.query(Question).get(2)
a = Answer(question=q, content="네 자동으로 생성됩니다.", create_date=datetime.now())
db.add(a)
db.commit()

In [11]:
a.question

In [12]:
q.answer

In [5]:
for i in range(300):
    q = Question(subject="test data : [%03d]" % i, content=f"content {i}", create_date=datetime.now())
    db.add(q)

In [6]:
db.commit()

In [19]:
db = SessionLocal()

In [20]:
db.query(Question).count()

309

In [21]:
db.query(Answer).count()

8

In [22]:
db.query(Question).join(Answer).count()

7

In [23]:
db.query(Question).outerjoin(Answer).count()

312

In [24]:
db.query(Question).outerjoin(Answer).distinct().count()

309

In [40]:
db.query(Question).outerjoin(Answer).filter(
    Question.content.ilike("%파이썬%") |
    Answer.content.ilike("%파이썬%")
).distinct().count()

1

In [46]:
db.query(Answer.question_id, Answer.content, User.username).outerjoin(User, Answer.user_id == User.id).subquery()

<sqlalchemy.sql.selectable.Subquery at 0x107fde640; anon_1>

In [47]:
sub_query = db.query(Answer.question_id, Answer.content, User.username) \
.outerjoin(User, Answer.user_id == User.id).subquery()

In [49]:
sub_query.c.question_id

Column('question_id', Integer(), ForeignKey('question.id'), table=<anon_1>)

In [51]:
db.query(Question).outerjoin(sub_query, sub_query.c.question_id == Question.id) \
    .filter(sub_query.c.content.ilike("%파이썬%") | 
            sub_query.c.username.ilike("%파이썬%")).distinct().count()

0

In [53]:
db.query(Question).outerjoin(sub_query, sub_query.c.question_id == Question.id) \
    .filter(sub_query.c.content.ilike('%파이썬%') |   # 답변내용
           sub_query.c.username.ilike('%파이썬%')    # 답변작성자
           ).distinct().count()

0